In [ ]:
import psycopg2
import pandas as pd
import sys
import os

path = '/Users/tranncl/SpicedDataScience/MovieRecommender'
os.chdir(path)

def pg_load_table(file_path, table_name, dbname, host, port, user, pwd):
    '''
    This function upload csv to a target table
    '''
    try:
        conn = psycopg2.connect(dbname=dbname, host=host, port=port,\
         user=user, password=pwd)
        print("Connecting to Database")
        cur = conn.cursor()
        f = open(file_path, "r")
        # Truncate the table first
        cur.execute("Truncate {} Cascade;".format(table_name))
        print("Truncated {}".format(table_name))
        # Load table from the file with header
        cur.copy_expert("copy {} from STDIN CSV HEADER QUOTE '\"'".format(table_name), f)
        cur.execute("commit;")
        print("Loaded data into {}".format(table_name))
        conn.close()
        print("DB connection closed.")

    except Exception as e:
        print("Error: {}".format(str(e)))
        sys.exit(1)

        
# title.ratings.tsv – Contains the IMDb rating and votes information for titles
# tconst (string) - alphanumeric unique identifier of the title
# averageRating – weighted average of all the individual user ratings
# numVotes - number of votes the title has received
def create_table_rating():
    conn=psycopg2.connect("dbname=movies user=tranncl password=tranncl host=localhost port=5432")
    cur=conn.cursor()
    cur.execute("CREATE TABLE rating \
                    (id TEXT, averageRating FLOAT, numVotes INTEGER)"\
               )
    conn.commit()
    conn.close()

    
# title.akas.tsv - Contains the following information for titles:
# titleId (string) - a tconst, an alphanumeric unique identifier of the title
# ordering (integer) – a number to uniquely identify rows for a given titleId
# title (string) – the localized title
# region (string) - the region for this version of the title
# language (string) - the language of the title
# types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
# attributes (array) - Additional terms to describe this alternative title, not enumerated
# isOriginalTitle (boolean) – 0: not original title; 1: original title

def create_table_akas():
    conn=psycopg2.connect("dbname=movies user=tranncl password=tranncl host=localhost port=5432")
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS akas (\
                    titleId CHAR(10), ordering SMALLINT, title TEXT, \
                    region VARCHAR(30), language VARCHAR(30), types TEXT, attributes TEXT, \
                    isOriginalTitle CHAR(3))"\
               )
    conn.commit()
    conn.close()

    
# title.basics.tsv.gz - Contains the following information for titles:
# tconst (string) - alphanumeric unique identifier of the title
# titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
# primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
# originalTitle (string) - original title, in the original language
# isAdult (boolean) - 0: non-adult title; 1: adult title
# startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
# endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
# runtimeMinutes – primary runtime of the title, in minutes
# genres (string array) – includes up to three genres associated with the title   

def create_table_title_basics():
    conn=psycopg2.connect("dbname=movies user=tranncl password=tranncl host=localhost port=5432")
    cur=conn.cursor()
    cur.execute("CREATE TABLE title_basics ( \
                    id CHAR(10), titleType VARCHAR(30), primaryTitle TEXT, \
                    originalTitle TEXT, isAdult CHAR(10), startYear CHAR(5), endYear CHAR(5), \
                    runtimeMinutes TEXT, genres TEXT)" \
               )
    conn.commit()
    conn.close()

# Convert tab separator file to csv

def convertToCsv(inputtabfile, outputcsvfile):
    df = pd.read_csv(path+inputtabfile, delimiter='\t')
    df.to_csv(path+outputcsvfile, index=False)

def insert_movies_rating():
    conn=psycopg2.connect("dbname=movies user=tranncl password=tranncl host=localhost port=5432")
    cur=conn.cursor()
    cur.execute("SELECT A.titleid, A.ordering, A.title, A.region, A.isoriginaltitle, A.language, A.types, \
                    A.attributes, B.averagerating, B.numvotes \
                INTO movies_rating \
                FROM akas A, rating B \
                WHERE A.titleid = B.id")
    conn.commit()
    conn.close()

def insert_movies():
    conn=psycopg2.connect("dbname=movies user=tranncl password=tranncl host=localhost port=5432")
    cur=conn.cursor()
    cur.execute("SELECT A.titleid, A.ordering, A.title, A.region, A.isoriginaltitle, A.language, A.types, \
                    A.attributes, A.averagerating, A.numvotes, B.titletype, B.primarytitle, B.originaltitle, \
                    B.isadult, B.startyear, B.endyear, B.runtimeminutes, B.genres \
                 INTO movies \
                 FROM movies_rating A, title_basics B \
                 WHERE A.titleid = B.id")
    conn.commit()
    conn.close()

In [ ]:
convertToCsv("title.ratings.tsv", "title.ratings.csv")
convertToCsv("title.basics.tsv", "title.basics.csv")
convertToCsv("title.akas.tsv", "title.akas.csv")

In [ ]:
# Insert csv files
dbname = 'movies'
host = 'localhost'
port = '5432'
user = 'tranncl'
pwd = 'tranncl'

# Insert table title_basics
file_path = path + "title.basics.csv"
table_name = 'title_basics'
pg_load_table(file_path, table_name, dbname, host, port, user, pwd)

# Insert table rating
file_path = path + "title.ratings.csv"
table_name = 'rating'
pg_load_table(file_path, table_name, dbname, host, port, user, pwd)

# Insert table akas
file_path = path + "title.akas.csv"
table_name = 'akas'
pg_load_table(file_path, table_name, dbname, host, port, user, pwd)

In [ ]:
insert_movies_rating()
insert_movies()